##  Calculate Yearly DPLE variables

In [1]:
import xarray as xr
import numpy as np
import metabolic as mi
import os
import pandas as pd
import utils
import warnings
warnings.filterwarnings('ignore')

In [2]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """

    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)

In [3]:
USER = os.environ['USER']

In [34]:
#varr = 'O2'
#varr = 'AOU'
#varr = 'T'
varr = 'S'

In [35]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_vertical_mean_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)

In [36]:
dsf = xr.Dataset()
for cc in range(3):
    print(cc)
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{varr}_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{varr}_{cc}_{layer}'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')

0
1
2


In [37]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:           (nlat: 384, nlon: 320, year: 70)
Coordinates:
    TLAT              (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.19 72.19
    TLONG             (nlat, nlon) float64 320.6 321.7 322.8 ... 319.4 319.8
    ULAT              (nlat, nlon) float64 -78.95 -78.95 -78.95 ... 72.41 72.41
    ULONG             (nlat, nlon) float64 321.1 322.3 323.4 ... 319.6 320.0
  * year              (year) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Dimensions without coordinates: nlat, nlon
Data variables:
    Phi_S_0_0-200m    (nlat, nlon, year) float64 nan nan nan nan ... nan nan nan
    Phi_S_0_200-600m  (nlat, nlon, year) float64 nan nan nan nan ... nan nan nan
    Phi_S_1_0-200m    (nlat, nlon, year) float64 nan nan nan nan ... nan nan nan
    Phi_S_1_200-600m  (nlat, nlon, year) float64 nan nan nan nan ... nan nan nan
    Phi_S_2_0-200m    (nlat, nlon, year) float64 nan nan nan nan ... nan nan nan
    Phi_S_2_200-600m  (nlat, nlon, year) float64 nan nan nan nan ... nan nan nan

In [38]:
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_vertical_mean_ens_mean_monthly_to_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_vertical_mean_1948-2017_monthly_to_yearly.nc', mode='w')

## LME monthly to yearly

In [39]:
path = f'/glade/scratch/{USER}/DPLE-results'
#filename = f'{path}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly.nc'
filename = f'{path}/DPLE-FOSI_hindcast_Phi_components_{varr}_LME_1948-2017_monthly.nc'
ds = xr.open_dataset(filename)
ds

<xarray.Dataset>
Dimensions:           (lme: 65, time: 840)
Coordinates:
  * lme               (lme) int64 1 2 3 4 5 6 7 8 9 ... 58 59 60 61 62 63 64 65
  * time              (time) object 0249-02-01 00:00:00 ... 0319-01-01 00:00:00
Data variables:
    Phi_S_0_0-200m    (lme, time) float64 ...
    Phi_S_0_200-600m  (lme, time) float64 ...
    Phi_S_1_0-200m    (lme, time) float64 ...
    Phi_S_1_200-600m  (lme, time) float64 ...
    Phi_S_2_0-200m    (lme, time) float64 ...
    Phi_S_2_200-600m  (lme, time) float64 ...

In [40]:
dsf = xr.Dataset()
for cc in range(3):
    print(cc)
    for layer in ['0-200m', '200-600m']:
        dsf[f'Phi_{varr}_{cc}_{layer}'] = xr_reshape(ds[f'Phi_{varr}_{cc}_{layer}'], 'time', ['year', 'mon'], [np.arange(70), np.arange(12)]).mean(dim='mon')

0
1
2


In [41]:
dout = f'/glade/scratch/{USER}/DPLE-results'
os.makedirs(dout, exist_ok=True)
dsf.load()

<xarray.Dataset>
Dimensions:           (lme: 65, year: 70)
Coordinates:
  * lme               (lme) int64 1 2 3 4 5 6 7 8 9 ... 58 59 60 61 62 63 64 65
  * year              (year) int64 0 1 2 3 4 5 6 7 8 ... 62 63 64 65 66 67 68 69
Data variables:
    Phi_S_0_0-200m    (lme, year) float64 0.0002084 0.0004295 ... 0.0003802
    Phi_S_0_200-600m  (lme, year) float64 -1.311e-05 1.53e-07 ... 4.695e-06
    Phi_S_1_0-200m    (lme, year) float64 0.0002626 0.0005482 ... 0.0004669
    Phi_S_1_200-600m  (lme, year) float64 -1.64e-05 9.604e-08 ... 5.768e-06
    Phi_S_2_0-200m    (lme, year) float64 0.0003802 0.00081 ... 0.0006485
    Phi_S_2_200-600m  (lme, year) float64 -2.344e-05 -6.341e-08 ... 8.004e-06

In [42]:
#dsf.to_netcdf(f'{dout}/DPLE_driftcorrected_Phi_LME_ens_mean_monthly_to_yearly.nc', mode='w')
dsf.to_netcdf(f'{dout}/DPLE-FOSI_hindcast_Phi_components_{varr}_LME_1948-2017_monthly_to_yearly.nc', mode='w')